In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install --quiet lightning
! pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 15.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import string
from sklearn.model_selection import train_test_split
import transformers
from torch.utils.data import DataLoader, Dataset
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

## with Meta Features

In [4]:
# Define the parameters to log with MLflow
params = {
    "train_path": "/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/train_preprocessed_v2.csv",
    "test_path": "/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/new_essay_val_preprocessed.csv",
    "test_path_dep": "/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/test_data_preprocessed.csv",
    "num_labels": 2,
    "batch_size": 6,
    "lr": 2e-5,
    "epochs": 3
}

In [5]:
#Loading preprocessed Train and Test Data
train_data_processed = pd.read_csv(params["train_path"])
test_data_processed = pd.read_csv(params["test_path"])
test_data_dep_processed = pd.read_csv(params["test_path_dep"])

In [6]:
display(train_data_processed)

,text,label,sentence_variation,%stop_word_total,mean_word_length,%adjectives_total
0,school homework clubs become increasingly popu...,1.0,5.291503,0.404908,5.398773,0.055215
1,widely accepted knowledge great source power s...,1.0,6.047842,0.459930,5.236934,0.111498
2,first impressions great power shape interactio...,1.0,9.306841,0.460784,5.250000,0.117647
3,name address city state zip code email address...,1.0,14.740441,0.408935,5.567010,0.096220
4,limiting car usage numerous advantages benefit...,1.0,6.080817,0.404959,5.524793,0.090909
...,...,...,...,...,...,...
28733,agathla peak agathlan mountain peak located we...,1.0,9.130170,0.449275,5.014493,0.043478
28734,hans schinz 6 december 1858 – 30 november 1915...,1.0,8.870957,0.366071,4.660714,0.035714
28735,pilgrimage route kii peninsula japan connectin...,1.0,3.696846,0.416000,4.976000,0.016000
28736,musica sacra magazine sacred music founded 197...,1.0,7.979263,0.424390,5.131707,0.121951


In [7]:
train_data_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28738 entries, 0 to 28737
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   text                28737 non-null  object 
 1   label               28738 non-null  float64
 2   sentence_variation  28738 non-null  float64
 3   %stop_word_total    28738 non-null  float64
 4   mean_word_length    28738 non-null  float64
 5   %adjectives_total   28738 non-null  float64
dtypes: float64(5), object(1)
memory usage: 1.3+ MB


In [8]:
test_data_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   text                40 non-null     object 
 1   label               40 non-null     int64  
 2   sentence_variation  40 non-null     float64
 3   %stop_word_total    40 non-null     float64
 4   mean_word_length    40 non-null     float64
 5   %adjectives_total   40 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.0+ KB


In [9]:
test_data_dep_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   text                50 non-null     object 
 1   label               50 non-null     int64  
 2   sentence_variation  50 non-null     float64
 3   %stop_word_total    50 non-null     float64
 4   mean_word_length    50 non-null     float64
 5   %adjectives_total   50 non-null     float64
dtypes: float64(4), int64(1), object(1)
memory usage: 2.5+ KB


In [10]:
# Split the dataset
df_train, df_validation = train_test_split(train_data_processed, test_size=0.3, random_state=42)
df_test = test_data_processed
df_test_dep = test_data_dep_processed

# Drop NA values
df_train.dropna(inplace=True)
df_validation.dropna(inplace=True)
df_test.dropna(inplace=True)
df_test_dep.dropna(inplace=True)

In [11]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [12]:
class TextMetaDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

        meta_features = dataframe[['sentence_variation', '%stop_word_total', 'mean_word_length', '%adjectives_total']]
        self.scaler = StandardScaler()
        self.scaler.fit(meta_features)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        record = self.dataframe.iloc[idx]
        text = record['text']
        label = record['label']
        metafeatures = [
            record['sentence_variation'],
            record['%stop_word_total'],
            record['mean_word_length'],
            record['%adjectives_total']
        ]

        metafeatures = self.scaler.transform([metafeatures])[0]

        # Tokenize the text
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Extract input_ids and attention_mask
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        metafeatures = torch.tensor(metafeatures, dtype=torch.float)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'meta_features': metafeatures,
            'label': torch.tensor(label, dtype=torch.long)
        }

In [13]:
# Create datasets and dataloaders
train_dataset = TextMetaDataset(df_train, tokenizer)
val_dataset = TextMetaDataset(df_validation, tokenizer)
test_dataset = TextMetaDataset(df_test, tokenizer)
test_dep_dataset = TextMetaDataset(df_test_dep, tokenizer)

train_loader = DataLoader(dataset=train_dataset, batch_size=params["batch_size"], shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=params["batch_size"])
test_loader = DataLoader(dataset=test_dataset, batch_size=1)
test_loader_dep = DataLoader(dataset=test_dep_dataset, batch_size=1)

In [14]:
class LightningModule(pl.LightningModule):
    def __init__(self, model_name, config, num_labels=2):
        super().__init__()
        # Initialize the XLNet model for sequence classification
        self.model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

        self.meta_feature_processor = nn.Sequential(
            nn.Linear(4, 16),  # 4 metafeatures
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU()
        )

        combined_features_dim = self.model.config.hidden_size + 16  # Assuming the hidden size from XLNet and meta features
        self.classifier = nn.Linear(combined_features_dim, num_labels)

    def forward(self, input_ids, attention_mask, meta_features):

        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.hidden_states[-1]
        sequence_features = last_hidden_state[:, 0, :]   # Get the features from the CLS token

        # Process metafeatures through the feed-forward network
        meta_features = self.meta_feature_processor(meta_features)


        combined_features = torch.cat((sequence_features, meta_features), dim=1)

        # Pass the combined features through the classifier
        logits = self.classifier(combined_features)
        return logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        meta_features = batch['meta_features']
        labels = batch['label']
        logits = self(input_ids, attention_mask, meta_features)
        loss = F.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        probs = torch.softmax(logits, dim=1)[:, 1]
        self.log('train_loss', loss, on_step=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        meta_features = batch['meta_features']
        labels = batch['label']
        logits = self(input_ids, attention_mask, meta_features)
        loss = F.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        probs = torch.softmax(logits, dim=1)[:, 1]
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=2e-5)
        return optimizer

In [15]:
config = transformers.DistilBertConfig.from_pretrained('distilbert-base-uncased', num_labels=2, output_hidden_states=True)
lightning_module = LightningModule('distilbert-base-uncased', config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lightning_module.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LightningModule(
  (model): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            

In [ ]:
callbacks = [
    ModelCheckpoint(
        save_top_k=1,
        mode="min",
        monitor="val_loss"
    )  # save top 1 model
]

# Trainer configuration
trainer = pl.Trainer(
    max_epochs=params["epochs"],
    callbacks=callbacks,
    devices=1,
    logger=False
)

trainer.fit(lightning_module, train_loader, val_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                   | Type                                | Params
-------------------------------------------------------------------------------
0 | model                  | DistilBertForSequenceClassification | 67.0 M
1 | meta_feature_processor | Sequential                          | 352   
2 | classifier             | Linear                              | 1.6 K 
-------------------------------------------------------------------------------
67.0 M    Trainable params
0         Non-trainable params
67.0 M    Total params
267.828 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
torch.save(lightning_module, '/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/models/DistilBert/DistilBert_kagglewiki_withmeta.pth')